# BIDDING STATERGIES

## Approach 1

In this approach, we just bid the predicted demand after subtracting the predicted solar output at the predicted market price.

In [1]:
import numpy as np
import pandas as pd
from scorer import bill_maker

In [2]:
demand=pd.read_csv("Prediction/demand.csv", header=None)
market_price=pd.read_csv("Prediction/market_price.csv", header=None)
solar_output=pd.read_csv("Prediction/solar_output.csv", header=None)

In [3]:
bid_quantity_pred=pd.DataFrame(demand-solar_output)

In [4]:
bid_price_pred=pd.DataFrame(market_price)

### Comparing prediction with Oracle

In [5]:
demand_oracle=pd.read_csv("Processed Data/Demand_Val_pred.csv", header=None)
market_price_oracle=pd.read_csv("Processed Data/Price_Val_pred.csv", header=None)
solar_output_oracle=pd.read_csv("Processed Data/Solar_Val_pred.csv", header=None)

In [6]:
bid_quantity_oracle=pd.DataFrame(demand_oracle - solar_output_oracle)

In [ ]:
bid_price_oracle=pd.DataFrame(market_price_oracle)

In [ ]:
daily_bill_pred,total_bill_pred=bill_maker(bid_price_pred,bid_quantity_pred)

In [ ]:
daily_bill_oracle,total_bill_oracle=bill_maker(bid_price_oracle,bid_quantity_oracle)

In [ ]:
total_bill_pred

In [ ]:
total_bill_oracle

### Saving prediction to output

In [ ]:
bid_quantity_pred.to_csv("Output/BidQuantity.csv", index=False, header=False)
bid_price_pred.to_csv("Output/BidPrice.csv", index=False, header=False)

## Approach 2

In this approach, we try to find the optimal margin that should be added to market price so that we avoid loosing the bid. We try to find this margin by considering the full training data.

In [ ]:
demand_actual_train=pd.read_csv("Processed Data/Demand_Train.csv", header=None)
market_price_train=pd.read_csv("Processed Data/Price_Train.csv", header=None)
market_price_train_oracle=pd.read_csv("Processed Data/Price_Train_pred.csv", header=None)
solar_output_train=pd.read_csv("Processed Data/Solar_Train.csv", header=None)

In [ ]:
pd.DataFrame(market_price_train.describe()[1:2].mean()).describe()

In [ ]:
pd.DataFrame(market_price.describe()[1:2].mean()).describe()

In [ ]:
pd.DataFrame(market_price.describe()[1:2].mean()).describe().loc[["mean"]]-pd.DataFrame(market_price_train.describe()[1:2].mean()).describe().loc[["mean"]]

From the above difference of mean, we can see that mean of predicted market price is greater than that of the training data, Hence we subtract this mean from the predicted market price.

In [ ]:
bid_price_pred=pd.DataFrame(market_price - 0.085249)

In [ ]:
daily_bill_pred,total_bill_pred=bill_maker(bid_price_pred,bid_quantity_pred)

### Comparing prediction with Oracle

In [ ]:
total_bill_pred

In [ ]:
total_bill_oracle

We can also find hour wise mean for the training data and correspondingly add/subtract from our predicted market price.

In [ ]:
hourly_mean=(market_price.describe().loc[["mean"]] - market_price_train.describe().loc[["mean"]]).mean()

In [ ]:
bid_price_pred=pd.DataFrame(market_price - hourly_mean)

In [ ]:
daily_bill_pred,total_bill_pred=bill_maker(bid_price_pred,bid_quantity_pred)

### Comparing prediction with Oracle

In [ ]:
total_bill_pred

In [ ]:
total_bill_oracle

We can also subtract the 25th percentile of the difference of price predicted by oracle and actual price when the predicted price by Oracle is less than Actual price (the case when we loose the bid) to ensure that 75% the we win it.

In [ ]:
margin_25=(market_price_train_oracle - market_price_train)[(market_price_train_oracle - market_price_train)<0].describe().loc[['25%']].mean()

In [ ]:
bid_price_pred=pd.DataFrame(market_price - margin_25)

In [ ]:
daily_bill_pred,total_bill_pred=bill_maker(bid_price_pred,bid_quantity_pred)

### Comparing prediction with Oracle

In [ ]:
total_bill_pred

In [ ]:
total_bill_oracle

### Saving prediction to output

In [ ]:
bid_quantity_pred.to_csv("Output/BidQuantity.csv", index=False, header=False)
bid_price_pred.to_csv("Output/BidPrice.csv", index=False, header=False)

## Approach 3

In this approach, we find the appropriate margin by considering the specific 50 days from the previous 2 years.

In [ ]:
market_price_train[-365:-365 + 50].values

In [ ]:
market_price_train[-365*2:-365*2 + 50].values

We find the average market price by the considering the 50 day period from the two previous years.

In [ ]:
market_price_avg = (market_price_train[-365:-365 + 50].values + market_price_train[-365*2:-365*2 + 50].values)/2

In [ ]:
market_price_avg=pd.DataFrame(market_price_avg)

In [ ]:
market_price_avg.head()

We model margin through subtracting the 25 percentile value of the data when predicted market price is less than average market price. 

In [ ]:
margin_25=(market_price - market_price_avg)[(market_price - market_price_avg)<0].describe().loc[['25%']].mean()

In [ ]:
bid_price_pred=pd.DataFrame(market_price - margin_25)

In [ ]:
daily_bill_pred,total_bill_pred=bill_maker(bid_price_pred,bid_quantity_pred)

### Comparing prediction with Oracle

In [ ]:
total_bill_pred

In [ ]:
total_bill_oracle

We can also add the absolute margin to the predicted market price.

In [ ]:
abs_margin=(market_price - market_price_avg).abs().describe().loc[["mean"]].mean()

In [ ]:
bid_price_pred=pd.DataFrame(market_price + abs_margin)

In [ ]:
daily_bill_pred,total_bill_pred=bill_maker(bid_price_pred,bid_quantity_pred)

### Comparing prediction with Oracle

In [ ]:
total_bill_pred

In [ ]:
total_bill_oracle